In [2]:
pip install langchain-core langgraph>0.2.27

    PyYAML (>=5.1.*)
            ~~~~~~^
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -U langchain-anthropic

    PyYAML (>=5.1.*)
            ~~~~~~^
  Attempting uninstall: langchain-core━━━━━━━━━━ 0/3 [anthropic]
    Found existing installation: langchain-core 0.3.600/3 [anthropic]
    Uninstalling langchain-core-0.3.60:━━━━━ 0/3 [anthropic]
      Successfully uninstalled langchain-core-0.3.602m0/3 [anthropic]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [langchain-anthropic]ngchain-anthropic]
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

  Using cached aiosqlite-0.21.0-py3-none-any.whl.metadata (4.3 kB)
Using cached aiosqlite-0.21.0-py3-none-any.whl (15 kB)
    PyYAML (>=5.1.*)
            ~~~~~~^
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [langgraph-checkpoint-sqlite]checkpoint-sqlite]
Note: you may need to restart the kernel to use updated packages.


In [7]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

········


In [8]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

········


In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Monday, June 2, 2025. San Francisco, CA - Weather Forecast', 'url': 'https://weathershogun.com/weather/usa/ca/san-francisco/480/june/2025-06-02', 'content': 'San Francisco, California Weather: Monday, June 2, 2025. Cloudy weather, overcast skies with clouds. Day 63°. Night 52°. Precipitation 25 %.', 'score': 0.95100147}, {'title': 'Weather in San Francisco in February 2025 (California)', 'url': 'https://world-weather.info/forecast/usa/san_francisco/february-2025/', 'content': '*   [6 +59°+46°](https://world-weather.info/forecast/usa/san_francisco/06-february/)\n*   [7 +55°+52°](https://world-weather.info/forecast/usa/san_francisco/07-february/)\n*   [8 +55°+46°](https://world-weather.info/forecast/usa/san_francisco/08-february/)\n*   [9 +55°+45°](https://world-weather.info/forecast/usa/san_francisco/09-february/)\n*   [10 +55°+43°](https://world-weather.info/forecast/usa/san_francisco/10-february/) [...] Average weather in February 2025\n--------------------------------\n\n

In [10]:
pip install -qU "langchain[google-genai]"

    PyYAML (>=5.1.*)
            ~~~~~~^
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.26.0 requires numpy<2,>=1.19.3, but you have numpy 2.2.6 which is incompatible.
streamlit 1.26.0 requires protobuf<5,>=3.20, but you have protobuf 6.31.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4", model_provider="openai")

Enter API key for OpenAI: ········


In [12]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [13]:
model_with_tools = model.bind_tools(tools)

In [14]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?

ToolCalls: []


In [15]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_B34LVPM9E52YbTRgCIk6AjLE', 'type': 'tool_call'}]


In [16]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [17]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='cca23801-c95c-4045-97fd-e5d6d27b83b3'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BdqXAhU4CPcTX2C3MNZ4tRaC5fUNR', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d4275484-2f11-41ae-9270-6f45e63ecc07-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [18]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='30e0e5b0-4956-479a-b358-2904e18c87e1'),
 AIMessage(content="Sorry, I can't provide real-time information like current weather updates. You can check the weather on a reliable website or app such as the National Weather Service, BBC Weather, or a local news website.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 88, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BdqXV3V3ywIBexpJni0KckD3Ww4k5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8722696d-5a40-4fd6-a4ec-2ffce3de3794-0', usage_metadata={'input_tokens': 88, 'output_tokens': 42, 'total_

In [19]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_PX24fbGFvBJnwey6BhaKDLyg)
 Call ID: call_PX24fbGFvBJnwey6BhaKDLyg
  Args:
    query: current weather in San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Monday, June 2, 2025. San Francisco, CA - Weather Forecast", "url": "https://weathershogun.com/weather/usa/ca/san-francisco/480/june/2025-06-02", "content": "San Francisco, California Weather: Monday, June 2, 2025. Cloudy weather, overcast skies with clouds. Day 63°. Night 52°. Precipitation 25 %.", "score": 0.92343384}, {"title": "February 2025 Weather History in San Francisco California, United ...", "url": "https://weatherspark.com/h/m/557/2025/2/Historical-Weather-in-February-2025-in-San-Francisco-California-Un

In [20]:
 for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| current| weather| in| San| Francisco| is| cloudy| with| over|cast| skies|.| The| temperature| is| around| |65|°F| during| the| day| and| it| will| drop| to| around| |52|°F| at| night|.| There| is| a| |25|%| chance| of| precipitation|.| [|source|](|https|://|we|athers|hog|un|.com|/weather|/|usa|/ca|/s|an|-fr|anc|isco|/|480|/j|une|/|202|5|-|06|-|02|)|

In [21]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [22]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [23]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 85, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BdqZWjWqB9uQEW3LovUvo2iSa7G3h', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8e44ab39-ca42-4cb4-87aa-e0933739281c-0', usage_metadata={'input_tokens': 85, 'output_tokens': 11, 'total_tokens': 96, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [24]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. So, I don't know your name.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 86, 'total_tokens': 134, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BdqZtsltCE7mXXAEsJ85FGnL5WLGM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a0aa247a-5ffb-44df-bddd-e56574282f3b-0', usage_metadata={'input_tokens': 86, 'output_tokens': 48, 'total_tokens': 134, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio'